# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.313953488372093
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.2222222222222222
>>> Standard errors for PUB 0: 1.4996172247918447
>>> Expectation values for PUB 1: -0.01708984375
>>> Standard errors for PUB 1: 0.015752037506918087
>>> Expectation values for PUB 2: 0.13970588235294118
>>> Standard errors for PUB 2: 0.1307160956701727


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.0608189   1.00486071  0.83804693  0.60944046  0.29464245 -0.04296317
  -0.36624773 -0.64259105 -0.8812753  -1.02448586 -1.06108411 -0.99690456
  -0.84149459 -0.61209251 -0.29384684  0.04004591  0.35007024  0.62455713
   0.88498817  1.03482884  1.06426656]
 [ 0.0506541   0.34503135  0.63092204  0.86615863  1.00247386  1.05339317
   1.02209901  0.85661126  0.6229659   0.29543806 -0.01750351 -0.36359568
  -0.64656912 -0.86271097 -1.02369024 -1.06744902 -1.0098996  -0.84096418
  -0.59193695 -0.30021175  0.02148158]
 [-0.0617927  -0.37367346 -0.61766181 -0.85555044 -1.00220866 -1.04225457
  -0.96826245 -0.82054342 -0.59087613 -0.29915093  0.02599006  0.34131848
   0.62482233  0.84653348  1.00220866  1.04145896  0.98550076  0.84043378
   0.59856707  0.2919904  -0.04667603]
 [ 1.03323761  0.96932327  0.8306212   0.606258    0.31161555 -0.02545965
  -0.34025766 -0.63861298 -0.85979372 -0.99478293 -1.0435806  -0.99292649
  -0.81709576 -0.59379338 -0.29411204  0.0267

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: -0.6176470588235294
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.038461538461538464
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['0110101110001010110010111001000100001010111101111101000101101010100001010001010010010010000110100101010111100110010001000110001', '1010110000100111111111000111010101001100011100011010000000001011101000010000000100010000101011010101000000100100100000000010110', '0110101011000011010011101110000001100000101000010101110001011001011000001001000000010111101100010000001100000010101101110111010', '0111111100110010000010010000010001000100110001111111000000010000010010111010000011111111001110110001000100100101000100000111100', '1100000101011000110011100110011101001000101000100011101001111001110001100011110001111001110010110101000101111010000000111010011', '0101110100001101110101101110100001110000011001111101000100001011101001000010110001010001000010011110010000110000000001110001101', '1000100010101011111110101100010000000100000000101010110101100010101000010001111010010011100010001101010000110010010001000000001', '01001100001100111101001100100010011001100010010110100101001

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['1001001111010010110000000000000000000110001001000000000001101000000001101001101011010000001010000001010110001100011000000100110', '0011100001001001010110000011101101010101011111011010100100111000011110001000110101000011000110000010001011010100010001000101111', '1111000011010001010001001101010111001110110001001111001000000001011001100011111111101000001011001000001010000001010010100100101', '0011101100110010100111010111010000010111001000010100001000001001100000010010010011100011000100110111011010110001000010000100000', '1010111100100100100111001001011110100001011011000000100001101010001011010110011011011100101100100000000000001000000000000001001', '0111010110000000001100010011101010011110001111000010011110100110110100101100100001110010101010010110010001010000000000000100001', '0111011111111001011110000001101011110101010010000101100110101101101101101001101011111111010110100000000001000000110000011100011', '0011100010001110000001110001101111011000000010000

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0101100001111110110001100100111111100010001010101001000010011101101010110101100000010000100011010101100000111010111001011101110', '0001011011110010000011100111111100010111101011110011111000101001010111101111011001101000101100110011110100110010001001100000001', '0110011001111101110101100100110100010000110001101011110000101110001011110100100010000111110010111000100100110100001110110000110', '0000111011110001000010000100110100101000011001001011101100111000110110101101101100001000110111011000001001001100010111010011001', '0010010011011011011010011101111000100001111010001001111110111110000110110110001110001011101100110110101100101011100010010011101', '1111001001011110110011000110111010110101100111010100001001001001010010100100100010100010110101010010111010001111100110111000101', '0101001011010101101101010100011110100000000100011001000000101000000000100011011010010010110010101101110101011100010100010110101', '00000100010101111110010011110

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [8]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1100100101001101010001010001110000100101110111101011110101000110111000110000001000100001100011000001001111011101100001000100111', '0000101111110000100000001101110100100000010100001010001101001010011101010000100001001000000000000011000111010011100010011001001', '1000110110111011100001000011110001000110110110110100100100101110000101110101001000001001101110010010110010100000110010001001010', '1010000000101011101111100111000110110000110010111001111100100000101101011011100000010011000010001010111100010110101001011011100', '1000010001000100010010000011110000101001010011011001000001010101001100100010100110001000001101011001001100011101111001010101011', '0101001100000001000011010110010101001011000101001001001001101100111111001010001001110000000000010001100011110000110000000001100', '0000111001010111010111000011110100110000100100000000101001110100001100001000100000100000000010101010111010101010000001010000111', '1001011010001111110110000011111100

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1001000001001010011100010000000010110011100010011110111001010110101100101101011001100110000100111100010000001100100000000100001', '0011111100110111110000101110010111011001110010001110011100110000010110011110000001110100100101010100100110000100100000010100000', '1010110100000010110100010011101010110100101101101000010111110000101100000110010110001010111100001111001110010101000101000100001', '1000111100000000001101011001111010000101101110111010011001010100100000000000000001011110111000110000001010000100101001010100000', '0101011101111001110110101111100000110100001000101010010100110111100100100000000001011010001000010100001110011000000110000010100', '0001000010011010100001011001000011110100001100111010110000100011000110010101100010101111100001000000000101011001100001010000011', '0101101111000000010111110010011011000100000111010000111010010001101101010010100101000111000100100001111100010000000001000000101', '0001001001101011110101110111010110

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>